In [34]:
is_mac = False

if is_mac == True:
    baseprompt_filepath = "Data/basePrompt.in"
    instagram_data_filepath = "Data/instagram_raw.csv"
    output_file = "./public/events.json"
    image_filepath = './public/InstagramImages/'
else:
    baseprompt_filepath = "Data/basePrompt.in"
    instagram_data_filepath = "Data/instagram_raw.csv"
    output_file = "../public/events.json"
    image_filepath = "../public/InstagramImages/"


In [ ]:
import pandas as pd
import datetime 
import time
from requests.exceptions import RequestException
from typing import List
import json
from together import Together
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json
import pandas as pd
from enum import Enum, auto
import re
from pymongo import MongoClient, errors
from pymongo.server_api import ServerApi
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

basePrompt = open(baseprompt_filepath,"r", encoding = "utf-8").read()

load_dotenv()
togetherAPI = os.getenv('TOGETHER_API')
#Updating WUSA Events
webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

WUSAjson = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

wusaDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
wusaDf = pd.DataFrame(columns = postscolumns)

import time

time_now = int(time.time())
for event in WUSAjson:
    if int(datetime.datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print("No street address", str(err))
            location = None
        try:
            new_row = pd.DataFrame({
                    "account": ["WUSA"],
                    "date": [time_now],
                    "caption": str(event["description"])+" [For More Information, Click View Post] ",
                    "display_photo": event["image"],
                    "url": [event['url']],
                    "likes": [0],
                    "event_details": [{
                        "is_event": True,
                        "event_name": event["name"],
                        "event_description": str(event["description"])+" ... ",
                        "categories": ["SOCIAL"],
                        "start_time": int(datetime.datetime.fromisoformat(event["startDate"]).timestamp()),
                        "end_time": int(datetime.datetime.fromisoformat(event["endDate"]).timestamp()),
                        "location": location,
                    }]
            })
            wusaDf = pd.concat([wusaDf, new_row])
            print("success")
        except Exception as error:
            print("Failed row, error:", error)

Failed row, error: 'image'
success
success
success
success
success
success
success
success
success
success
success
Failed row, error: 'image'
success
No street address 'streetAddress'
Failed row, error: 'image'
success
No street address 'streetAddress'
success
No street address 'streetAddress'
Failed row, error: 'image'
No street address 'streetAddress'
success
success
No street address 'location'
success
Failed row, error: 'image'
success
No street address 'location'
success
success
success


In [16]:
together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

# Set up MongoDB client
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with 'account': "WUSA"
try:
    result = collection.delete_many({'account': "WUSA"})
    print(f"Documents removed: {result.deleted_count}")
except errors.PyMongoError as e:
    print(f"Error deleting documents: {e}")

# Check and remove '_id' column if present
if '_id' in wusaDf.columns:
    print("DataFrame contains an '_id' column. It will be removed to prevent duplication.")
    wusaDf = wusaDf.drop(columns=['_id'])

for index, row in wusaDf.iterrows():
    # Construct the info string
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embeddedtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    # Create the document dictionary
    row_dict = {column: row[column] for column in wusaDf.columns.tolist()}
    
    # Add the embedded field
    row_dict["embedded"] = generate_embedding([embeddedtext], embedding_model_string)
    
    # Remove '_id' if present to let MongoDB generate it
    row_dict.pop('_id', None)
    
    try:
        result = collection.insert_one(row_dict)
        print(f"Inserted document ID: {result.inserted_id}")
        insertObjectIds.append(result.inserted_id)
    except errors.DuplicateKeyError as e:
        print(f"DuplicateKeyError: {e}. Document skipped.")
    except errors.PyMongoError as e:
        print(f"An error occurred: {e}. Document skipped.")
    
    time.sleep(0.5)  # Adjust or remove delay as needed

# Optionally, print all inserted ObjectIds
print(f"All inserted document IDs: {insertObjectIds}")

Documents removed: 21
Inserted document ID: 6726cbf9fa90a11c6574e340
Inserted document ID: 6726cbfafa90a11c6574e341
Inserted document ID: 6726cbfbfa90a11c6574e342
Inserted document ID: 6726cbfbfa90a11c6574e343
Inserted document ID: 6726cbfcfa90a11c6574e344
Inserted document ID: 6726cbfdfa90a11c6574e345
Inserted document ID: 6726cbfefa90a11c6574e346
Inserted document ID: 6726cbfffa90a11c6574e347
Inserted document ID: 6726cc00fa90a11c6574e348
Inserted document ID: 6726cc00fa90a11c6574e349
Inserted document ID: 6726cc01fa90a11c6574e34a
Inserted document ID: 6726cc02fa90a11c6574e34b
Inserted document ID: 6726cc03fa90a11c6574e34c
Inserted document ID: 6726cc04fa90a11c6574e34d
Inserted document ID: 6726cc04fa90a11c6574e34e
Inserted document ID: 6726cc05fa90a11c6574e34f
Inserted document ID: 6726cc06fa90a11c6574e350
Inserted document ID: 6726cc07fa90a11c6574e351
Inserted document ID: 6726cc08fa90a11c6574e352
Inserted document ID: 6726cc09fa90a11c6574e353
Inserted document ID: 6726cc09fa90a11c

In [ ]:
#Validating if it is an event
load_dotenv()
togetherAPI = os.getenv('TOGETHER_API')
client = Together(api_key=togetherAPI)

postsDf = pd.read_csv(instagram_data_filepath).replace('"','', regex=True)
postsDf["is_event"] = pd.NA
postsDf["processed_json"]=pd.NA

def check_string(input_string):
    if any(word in input_string for word in ['yes', 'Yes', 'True', 'true']):
        return True
    elif any(word in input_string for word in ['no', 'No', 'False', 'false']):
        return False
    else:
        return False  # This handles cases where none of the words are found


cnt=0
for index, row in postsDf.iterrows():
    currentjson = f"'account': '{row['account']}'; 'caption': '{row['caption']}'; 'photo_caption': '{row['accessibility_caption']}; 'date_posted_on_instagram: {row['date']}'"
    postsDf.at[index, "processed_json"] = currentjson
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[{"role": "user", "content": f'Does the following instagram post contain a club event with a specified time. RETURN Yes or No only: {currentjson}'}],
        max_tokens=2
    )
    print(response.choices[0].message.content)
    is_event = check_string(response.choices[0].message.content)
    postsDf.at[index, "is_event"] = is_event
    time.sleep(0.6)

Yes
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
No
Yes
Yes
No
No


In [19]:
#Functions to Aid in LLM JSON data extraction


client = Together(api_key=togetherAPI)


def remove_emojis(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

class Category(Enum):
    TECH = auto()
    DESIGN = auto()
    SOCIAL = auto()
    ENTERTAINMENT = auto()
    CULTURE = auto()
    SPORTS = auto()
    NETWORKING = auto()
    GAMING = auto()

class Event(BaseModel):
    is_event: bool = Field(description="Whether the post contains an event")
    event_name: str = Field(description="The Name of the Event")
    event_description: str = Field(description='Concise 20 word summary of the event without time or location')
    event_categories: list[str] = Field(description='Categorize the Event into at least one or more of the following: TECH, DESIGN, SOCIAL, MUSIC, CULTURE, SPORTS, NETWORK, GAMING')
    start_time: str = Field(description="The Start time of Event in the format: yyyy-mm-ddTHH:MM:SS+00:00")
    end_time: str = Field(description="The End time of Event in the format: yyyy-mm-ddTHH:MM:SS+00:00")
    location: str = Field(description= "The location of event")

def return_event_details(inputJson : str):
    chat_completion = client.chat.completions.create(
    model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": Event.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": basePrompt,
        },
        {
            "role": "user",
            "content": "\n\n Input " + remove_emojis(inputJson) + "\n\n" + "Output",
        },
    ])
    
    created_event = json.loads(chat_completion.choices[0].message.content)
    return created_event

def extract_details_with_error_handling(inputJson, index):
        try: 
            created_event = return_event_details(inputJson)
            return created_event
        except Exception as err:
            print(str(err))
            return {'is_event': False, 'event_name': None, 'start_time': None, 'end_time': None, 'location': None}

postsDf["event_details"] = pd.NA


In [35]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os

def download_instagram_image(url, filepath):
    try:
        # Send a GET request to the Instagram post URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Attempt to find the high-resolution image URL
        image_meta = soup.find('meta', property='og:image')
        if image_meta and 'content' in image_meta.attrs:
            image_url = image_meta['content']
            
            # Modify the image URL to try to get the full-resolution version
            if 's150x150' in image_url:
                image_url = image_url.replace('s150x150', 's1080x1080')

            # Download the image
            image_response = requests.get(image_url)
            image_response.raise_for_status()

            # Create the directory if it doesn't exist
            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            # Save the image directly with the provided filepath
            with open(filepath, 'wb') as file:
                file.write(image_response.content)

            print(f"Image saved successfully: {filepath}")
            return True
        else:
            print("Could not find the image URL in the Instagram post.")
            return False

    except requests.RequestException as e:
        print(f"Error accessing the Instagram post or downloading the image: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


In [36]:
for index, row in postsDf.iterrows():
    if postsDf.at[index, "is_event"]== True:
        event_details = return_event_details(str(postsDf.at[index, "processed_json"]))
        print(index, event_details)
        if event_details["event_name"] == None or event_details["start_time"] == None or event_details["end_time"] == None:
            postsDf.at[index, "is_event"] = False
            postsDf.at[index, "event_details"] = None
        else:
            postsDf.at[index, "event_details"] = event_details
            imageurl = row["display_photo"]
            postID = row["url"]
            correcturl = f'https://www.instagram.com/p/{postID}/size=lg'
            filepath = image_filepath + postID + ".jpg"
            download_instagram_image(correcturl, filepath)
    else: print(index, "no event detected")

TypeError: 'Collection' object is not callable. If you meant to call the 'create' method on a 'Collection' object it is failing because no such method exists.

In [ ]:
#RAG Processing
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

#time conversion and edit formats
from datetime import datetime
import time

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")
postsDf.drop(postsDf.columns[postsDf.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = row["event_details"]
      try:
        conStart = datetime.fromisoformat(row_dict["start_time"])
        unixStart = time.mktime(conStart.timetuple()) 
        try:
          conEnd = datetime.fromisoformat(row_dict["end_time"])
          unixEnd = time.mktime(conEnd.timetuple())
        except:
          unixEnd = None
        row_dict["start_time"]=unixStart
        row_dict["end_time"]=unixEnd
      except:
        postsDf.at[index, "is_event"] = False
        err = f'Start failed {row_dict["start_time"]}'
        print(index, err)
      postsDf.at[index, "event_details"] = row_dict
      print(unixStart,unixEnd)

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      for column in postsDf.columns.tolist():
        if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
          row_dict[column] = row[column]
      row_dict["embedded"] = generate_embedding([row["processed_json"]], embedding_model_string)  
      row_dict["event_details"] = row["event_details"]
      result = collection.insert_one(row_dict)
      print(f"Inserted document ID: {result.inserted_id}")
      time.sleep(1)

#Updating Events.json

def download_future_events_to_json(output_file):
    # Get current Unix timestamp
    current_timestamp = int(time.time())

    # Connect to MongoDB
    load_dotenv()
    uri = os.getenv('DATABASE_URI')
    client = MongoClient(uri, server_api=ServerApi('1'))
    db = client['Instagram']
    collection = db["Events"]

    # Query for future events
    query = {}
    future_events = list(collection.find(query))

    # Convert ObjectId to string for JSON serialization
    for event in future_events:
        event['_id'] = str(event['_id'])

    # Write to JSON file
    with open(output_file, 'w') as f:
        json.dump(future_events, f, indent=2)

    # Close the MongoDB connection
    client.close()

    return future_events


future_events = download_future_events_to_json(output_file)

1731115800.0 1731123000.0
1730766600.0 1730770200.0
1731740400.0 1731826740.0
12 Start failed 2024-07-xxT20:00:00+00:00
1731740400.0 1731826740.0
1572913800.0 1572921000.0
15 Start failed 
1572913800.0 1572921000.0
Inserted document ID: 6726d06dfa90a11c6574e356
Inserted document ID: 6726d06efa90a11c6574e357
Inserted document ID: 6726d06ffa90a11c6574e358
Inserted document ID: 6726d071fa90a11c6574e359
